In [1]:
#from our_dataset import ProofDataset
from use_dataset import ProofDataset, HiddenConcProofDataset
#from our_dataset_with_labels import ProofDatasetWithLabels
from statement_embedding import emb_to_stmt, num_to_label
from torch_geometric.transforms import NormalizeFeatures
import numpy as np
import pandas as pd
import csv
import torch
from gensim.models import Word2Vec


In [2]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [3]:

file_limit = 5000    # desired number of graphs to work with
vocab_size = 1598   # number of characters in our vocabulary
label_size = file_limit  # number of labels in dataset; all labels is 45332; defaults to file_limit in pf_data

pf_data = pf_data = ProofDataset(root="data/",read_name="5000_relabeled_data_at_least_5.json" , write_name="10000_relabeled_data_at_least_5_w_stmts.pt" ,file_limit=file_limit)    

C:\Users\dphil\anaconda3\envs\erdos_may_2024\lib\site-packages\torch_geometric\data\dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.

In [750]:
#Gets the hypotheses for a given step
def hypos(cur_graph,step_num):
    req_stmt_num = []
    for idx, x in enumerate(cur_graph.edge_index[1]):
        if step_num == x:
            req_stmt_num.append(int(cur_graph.edge_index[0][idx]))
    return req_stmt_num

In [751]:
#Constructs a random walk up the graph of length less than 15
def graphwalks(cur_graph):
    walk=[cur_graph.num_nodes-1]
    start=cur_graph.num_nodes-1
    for i in range(15):
        hyp=hypos(cur_graph,start)
        if len(hyp)!=0:
            start=np.random.choice(hyp)
            walk.append(start)
    return walk

In [752]:
graphwalks(cur_graph)

[2, 1, 0]

In [753]:
#Creates a "sentence" based on a random graph walk. In hindsight, they should have been called paragraphs.
def sentence(cur_graph):
    walk=graphwalks(cur_graph)
    sent=[]
    for step_num in reversed(walk):
        cur_step = cur_graph.x[step_num]
        cur_step_vec = cur_step*vocab_size
        cur_step_vec = cur_step_vec[cur_step_vec.nonzero()] # remove trailing zeros
        cur_step_vec = cur_step_vec.reshape(-1)
        sent.extend(cur_step_vec.to(int).tolist())
        #The special character -1, acts as a sentence break, telling the model that a statement has ended
        sent.append(-1)
    return sent         

In [754]:
#Permutes the three most common symbolic variables. Necessary to prevent the model from trivializing
def perm(list,i):
    if i % 3==1:
        for j in range(len(list)):
            if list[j] == 114:
                list[j] = 1284
            elif list[j] == 1284:
                list[j] = 1181
            elif list[j] == 1181:
                list[j] = 114
    if i % 3==2:
        for j in range(len(list)):
            if list[j] == 114:
                list[j] = 1181
            elif list[j] == 1284:
                list[j] = 114
            elif list[j] == 1181:
                list[j] = 1284
    return list

In [755]:
#Creates 6 sentences for each graph up to 1058 (The end of the section on propostional logic).
sentences=[]
for i in range(2,1058):
    graph=pf_data.get(i)
    for j in range(6):
        if len(graph.edge_index.size()) != 1:
            sentences.append(perm(sentence(graph),i))
    

In [756]:
#constructs the dictionary of "words.
dict={857}
for sent in sentences:
    for word in sent:
        dict.add(word)

In [757]:
dictionary=list(dict)

In [758]:
length=len(dictionary)
print(length)

22


In [759]:
#Turns a word into a vector.
def tovec(word):
    vec=np.zeros(length)
    for i in range(length):
        if word==dictionary[i]:
            vec[i]=1
    return vec
            

In [760]:
#The length of the skipgrams used. The model will have difficulty with statements more than half the length here.
depth=40

In [761]:
#converts our sentences to vectors
vecsents=[]
for sent in sentences:
    for i in range(3):
        vecsent=[]
        start=np.random.randint(1,len(sent))
        truncsent=sent[0:start]
        if len(truncsent)<depth:
            for i in range(depth-len(truncsent)):
                vecsent.append(np.zeros(len(dictionary)))
        for word in truncsent:
            vecsent.append(tovec(word))
            if len(vecsent)==depth:
                break
    vecsents.append(np.array(vecsent))

In [762]:
datatens = torch.tensor(vecsents, dtype=torch.float32).reshape(len(vecsents), depth, length)

In [763]:
#Constructs the skip-grams as tensors
dataX=datatens[:,0:depth-1,:]
print(dataX.shape)
datay=datatens[:,depth-1,:]
print(datay.shape)

torch.Size([6252, 39, 22])
torch.Size([6252, 22])


In [764]:
#defines the model
hidden_layer=250
class predModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=length, hidden_size=hidden_layer, num_layers=1, batch_first=True)
        #self.trans = nn.Transformer(d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=70)
        self.dropout = nn.Dropout(0.05)
        self.linear = nn.Linear(hidden_layer, length)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:,-1]
        # produce output
        x = self.linear(self.dropout(x))
        return x

In [765]:
#Trains the model
n_epochs = 40
batch_size = 100
model = predModel()
 
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(dataX, datay), shuffle=True, batch_size=batch_size)
 
best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    loss = 0
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss += loss_fn(y_pred, y_batch)
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))


Epoch 0: Cross-entropy: 14179.5547
Epoch 1: Cross-entropy: 12063.4111
Epoch 2: Cross-entropy: 8721.8623
Epoch 3: Cross-entropy: 7737.4741
Epoch 4: Cross-entropy: 6972.9438
Epoch 5: Cross-entropy: 6717.7671
Epoch 6: Cross-entropy: 6681.5698
Epoch 7: Cross-entropy: 6267.8379
Epoch 8: Cross-entropy: 6144.9507
Epoch 9: Cross-entropy: 5996.0400
Epoch 10: Cross-entropy: 5838.7266
Epoch 11: Cross-entropy: 5647.4277
Epoch 12: Cross-entropy: 5527.3584
Epoch 13: Cross-entropy: 5332.2163
Epoch 14: Cross-entropy: 5276.4937
Epoch 15: Cross-entropy: 5108.1929
Epoch 16: Cross-entropy: 5066.8408
Epoch 17: Cross-entropy: 4996.2871
Epoch 18: Cross-entropy: 5211.0293
Epoch 19: Cross-entropy: 4638.4595
Epoch 20: Cross-entropy: 4462.8677
Epoch 21: Cross-entropy: 4338.2944
Epoch 22: Cross-entropy: 4304.8604
Epoch 23: Cross-entropy: 4193.3486
Epoch 24: Cross-entropy: 4020.1414
Epoch 25: Cross-entropy: 3921.9746
Epoch 26: Cross-entropy: 3695.0845
Epoch 27: Cross-entropy: 3687.5190
Epoch 28: Cross-entropy: 370

In [766]:
#creates an example for predicting the n-1 step of a graph. Most of this code is redoing everything we did to create the training data.
def createex(graph_num):
    sentences=[]
    cur_graph = pf_data.get(graph_num)
    for i in range(10):
        walk=[]
        sent=[]
        step=cur_graph.num_nodes-2
        for i in range(15):
            hyp=hypos(cur_graph,step)
            if len(hyp)!=0:
                step=np.random.choice(hyp)
                walk.append(step)
        for step_num in reversed(walk):
            cur_step = cur_graph.x[step_num]
            cur_step_vec = cur_step*vocab_size
            cur_step_vec = cur_step_vec[cur_step_vec.nonzero()] # remove trailing zeros
            cur_step_vec = cur_step_vec.reshape(-1)
            sent.extend(cur_step_vec.to(int).tolist())
            sent.append(-1)
        sentences.append(sent)
    vecsents=[]
    for sent in sentences:
        vecsent=[]
        if len(sent)<depth-1:
            for i in range(depth-1-len(sent)):
                vecsent.append(np.zeros(len(dictionary)))
        for word in sent[-(depth-1):]:
            vecsent.append(tovec(word))
        vecsents.append(vecsent)
    return vecsents

In [767]:
#Predicts the second to last step based on the example given
def predstat(example):
    predsentence=[]
    predletter=0
    i=0
    while predletter!=-1:
        exarray=np.array(example)
        extensor=torch.tensor(exarray[:,-(depth-1):,:], dtype=torch.float32).reshape(10, (depth-1),length)
        predletterraw = torch.sum(model(extensor), dim=0)
        predletterraw=nn.functional.normalize(predletterraw, dim=0)
        #print(predletterraw)
        predletter=dictionary[torch.argmax(predletterraw)]
        predsentence.append(predletter)
        for sent in example:
            sent.append(tovec(predletter));
    return predsentence

In [768]:
exnum=450
cur_graph = pf_data.get(exnum)
cur_step_vec = cur_graph.x[cur_graph.num_nodes-2]*vocab_size
cur_step_vec = cur_step_vec[cur_step_vec.nonzero()] # remove trailing zeros
cur_step_vec = cur_step_vec.reshape(-1)
cur_step_vec = cur_step_vec.to(int).tolist()
print("The actual statement is:",emb_to_stmt(cur_step_vec) ) 
print("The predicted statement is:",emb_to_stmt(predstat(createex(exnum))[:-1]) ) 

The actual statement is: ph -> ( ps -> ( ch -> th ) ) )
The predicted statement is: ( ( ph /\ ps ) /\ ch ) -> th ) -> ( ph -> ps ) )


It is difficult to quantify exactly how accurate this model is, since it typically cannot distinguish between the variables ph, ps, ch, etc.
The results have been typically "close" to the true answer, but, characteristic of deep learning models, it tends to switch similar symbols. Typically for shorter statements, the outputs have been genuine logical statements. However, for very long statements, the limited skip-gram length will lead to errors. (For technical reasons, the above output does not display opening parentheses.)


In [769]:
exnum=220
cur_graph = pf_data.get(exnum)
cur_step_vec = cur_graph.x[cur_graph.num_nodes-2]*vocab_size
cur_step_vec = cur_step_vec[cur_step_vec.nonzero()] # remove trailing zeros
cur_step_vec = cur_step_vec.reshape(-1)
cur_step_vec = cur_step_vec.to(int).tolist()
print("The actual statement is:",emb_to_stmt(cur_step_vec) ) 
print("The predicted statement is:",emb_to_stmt(predstat(createex(exnum))[:-1]) ) 

The actual statement is: ph -> ( ch <-> ps ) )
The predicted statement is: ph -> ( ps -> ch ) )


In [770]:
exnum=750
cur_graph = pf_data.get(exnum)
cur_step_vec = cur_graph.x[cur_graph.num_nodes-2]*vocab_size
cur_step_vec = cur_step_vec[cur_step_vec.nonzero()] # remove trailing zeros
cur_step_vec = cur_step_vec.reshape(-1)
cur_step_vec = cur_step_vec.to(int).tolist()
print("The actual statement is:",emb_to_stmt(cur_step_vec) ) 
print("The predicted statement is:",emb_to_stmt(predstat(createex(exnum))[:-1]) ) 

The actual statement is: ( ( th /\ ph ) /\ ps ) -> ch )
The predicted statement is: ( ph /\ ( ps /\ ch ) ) -> th )


In [771]:
#An example that behaves especially badly.
exnum=582
cur_graph = pf_data.get(exnum)
cur_step_vec = cur_graph.x[cur_graph.num_nodes-2]*vocab_size
cur_step_vec = cur_step_vec[cur_step_vec.nonzero()] # remove trailing zeros
cur_step_vec = cur_step_vec.reshape(-1)
cur_step_vec = cur_step_vec.to(int).tolist()
print("The actual statement is:",emb_to_stmt(cur_step_vec) ) 
print("The predicted statement is:",emb_to_stmt(predstat(createex(exnum))[:-1]) ) 

The actual statement is: th <-> ( ps /\ ch ) )
The predicted statement is: ( ph -> ps ) -> ( ph -> ( ph \/ ps ) ) )
